In [2]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_4


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings


loader = PyPDFLoader('../../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")



pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

chunks = pythontext_splitter.split_documents(documents)

print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")
print(type(chunks[0]))

pythonembeddings = UpstageEmbeddings(model="solar-embedding-1-large")


vectorstore = FAISS.from_documents(chunks, pythonembeddings)

print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

vectorstore.save_local("faiss_db")

pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        
    search_kwargs={"k": 5}          
    
)

pythonllm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.2
    )

pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],  # 템플릿 안에 사용된 변수명
    template=pythonprompt_template
)

pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,
    chain_type="stuff",
    retriever=pythonretriever,
    chain_type_kwargs={"prompt": prompt_template},  # ✅ 올바른 PromptTemplate 전달
    return_source_documents=True
)

pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【질문 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)

  총 109페이지 로드 완료
  104개 청크 생성 완료
  평균 청크 길이: 753자
<class 'langchain_core.documents.base.Document'>
 FAISS 벡터스토어 생성 완료 (104개 벡터)

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 사례

#### 1. **사례 기반 분석**  
제시된 사례집을 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 요소에 따라 결과가 달라집니다:  
- **계정 소유권 확인**: 계정 명의자와 실소유자가 다를 경우 복구가 거절될 수 있음(2006년 사례).  
- **오류 입증 가능성**: 시스템 오류가 객관적으로 확인되지 않거나, 다른 사용자에게 동일한 문제가 발생하지 않은 경우 복구 요구가 어려움(2009년 사례).  
- **회사의 복구 정책**: 프로그램 오류를 인정한 경우 복구가 가능했으나(2006년 사례), 약관상 책임 한계가 명시된 경우 거부될 수 있음(2007년 사례).  

#### 2. **법적 근거**  
- **민법 제250조(도품·유실물 특례)**:  
  - 아이템이 해킹 또는 시스템 오류로 소멸된 경우, 게임사는 "유실물 반환의무"를 질 수 있으나, **금전적 가치가 없는 디지털 아이템**은 적용 대상이 아닐 수 있음.  
  - 단, 아이템에 실제 금전적 투자(예: 현금 결제)가 있었다면 "재산적 가치"로 인정될 여지가 있음.  
- **전자상거래법 제17조(디지털콘텐츠 공급자의 의무)**:  
  - 시스템 오류로 인해 이용자가 피해를 입은 경우, 사업자는 **합리적인 조치**(복구, 보상 등)를 취해야 함.  
- **약관의 규제에 관한 법률 제6조(불